In [1]:
import os
import sys
import urllib.request
import pandas as pd
import json
from bs4 import BeautifulSoup
import re
import requests
from tabulate import tabulate

def BAR():
    print('--------------------------------------------------------------------')

def print_DF(df):
    print(tabulate(df, headers='keys', tablefmt='psql'))

#네이버 api를 통해 검색 후 상위 5개 검색어 반환
def searching(title):
    client_id = "JnkicuvSa2phrxPmgIXM"
    client_secret = "7nhnLrKQWp"
    
    encText = urllib.parse.quote(title)
    url = "https://openapi.naver.com/v1/search/book?query=" + encText +"&display=5"

    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    
    rescode = response.getcode()
    if(rescode == 200):
        response_body = response.read()
        result = response_body.decode('utf-8')
        result = json.loads(result)

        print("Success!")
        return result
    else:
        print("Error Code:" + rescode)
        return -1

#상위 5개의 검색 결과에서 사용자가 원하는 책을 고릅니다.
def choose_one_return_url(book):
    book_df = pd.DataFrame(book['items'])

    for i in range(len(book_df)):
        book_df.iloc[i]['title'] = BeautifulSoup(book_df.iloc[i]['title'], "html5lib").get_text()
    
    book_to_choice = book_df[['title','author']]

    print_DF(book_to_choice)   

    while 1:
        choice = int(input('원하는 책의 번호를 입력하세요: '))

        if choice < 0 or choice >= 5:
            print("Please enter 0 between 5.")
        else:
            break        
    print("Success!")

    return book_df.iloc[choice]["title"], book_df.iloc[choice]["link"]

#사용자가 선택한 책의 정보 전체를 받아옴
def make_description(URL):   
    url = requests.get(URL)

    #BeautifulSoup: requests로 받아온 html 문서를 보기 좋게 꾸며주는 틀
    soup = BeautifulSoup(url.text, 'html.parser')

    whole_review = soup.findAll("div", attrs={"id":"bookIntroContent"})
    whole_review

    for line in whole_review:
        book_des = line.get_text()
    print("Success!")
    return book_des

#한글 -> 영어
def translation(target):
    client_id = "GgPYL3vwgwG4djR0GQ0Y"
    client_secret = "CsLZxH0551"

    url = "https://openapi.naver.com/v1/papago/n2mt"

    #번역할 언어와 내용에 대해
    text = target
    encText = urllib.parse.quote(text)
    srcLang = 'ko'
    tarLang = 'en'
    data = "source={}&target={}&text=".format(srcLang,tarLang) + encText

    #웹 요청
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)

    #결과 받아오는 부분
    response = urllib.request.urlopen(request, data=data.encode("utf-8"))

    #응답
    rescode = response.getcode()
    if(rescode==200): #성공
        response_body = response.read()
        data = response_body.decode('utf-8')
        data = json.loads(data) #딕셔너리화
        trans_text = data['message']['result']['translatedText']
        trans_text = re.sub('[^A-Za-z0-9]+', ' ', trans_text)
        return trans_text
    else: #실패
        print("Error Code:" + rescode)
        return -1

In [2]:
result = searching(input("Please enter the title of the book : "))
book_title, url = choose_one_return_url(result)
book_des = make_description(url)

book_title = translation(book_title)
book_des = translation(book_des)

BAR()
print('책 제목: ', book_title)
print('책 줄거리: ', book_des)

Please enter the title of the book : 베이스볼
Success!
+----+----------------------------------------------------------------------+-----------------------+
|    | title                                                                | author                |
|----+----------------------------------------------------------------------+-----------------------|
|  0 | 스마트 베이스볼(Smart Baseball) (현대 야구를 지배하는 새로운 데이터) | 키스 로               |
|  1 | 주간베이스볼 週刊ベ-スボ-ル 2021.06.07                               | ベ-スボ-ル.マガジン社 |
|  2 | 주간베이스볼 週刊ベ-スボ-ル 2021.05.31                               | ベ-スボ-ル.マガジン社 |
|  3 | 주간베이스볼 週刊ベ-スボ-ル 2021.05.24                               | ベ-スボ-ル.マガジン社 |
|  4 | 주간베이스볼 週刊ベ-スボ-ル 2021.05.17                               | ベ-スボ-ル.マガジン社 |
+----+----------------------------------------------------------------------+-----------------------+
원하는 책의 번호를 입력하세요: 0
Success!
Success!
--------------------------------------------------------------------
책 제목:  Smart Baseball New Data Do

In [3]:
#주피터 노트북이 아닌 Colab에서 돌릴 경우 구글 드라이브에서 해당 파일 불러오는 부분
from google.colab import drive
drive.mount('/content/drive')

file = "/content/drive/MyDrive/Colab Notebooks/movies_metadata.csv"

data = pd.read_csv(file)

#Colab 아니면 아래 코드 이용
#data = pd.read_csv("movies_metadata.csv")

data = data.head(30000)

#영화 데이터 프레임에 책 description 추가
book_data = {'title': book_title, 'overview' : book_des}
data = data.append(book_data,ignore_index=True)


#Null 값 확인
data['overview'].isnull().sum()

# overview에서 NaN 값을 가진 경우를 제거(tf-idf를 할 때 NaN값이 들어있으면 에러가 나기 때문)
#해당 행을 그대로 두고 overivew의 NaN값만 제거
data['overview'] = data['overview'].fillna('')
#Null 값 확인
data['overview'].isnull().sum()

data['overview'].tail()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


29996    A beautiful lonely girl named Melissa tries to...
29997    A journalist starts an investigation into the ...
29998    When an out of work actress unexpectedly lands...
29999    Bumbling navy officer Lieutenant Humphrey Fair...
30000    Over the past decade the data revolution in th...
Name: overview, dtype: object

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english') # 불용어 제거

# overview에 대해서 TF-IDF 수행
tfidf_matrix = tfidf.fit_transform(data['overview'])
# (데이터 개수, 단어 개수)
print(tfidf_matrix.shape)



(30001, 58569)


In [5]:
from sklearn.metrics.pairwise import linear_kernel

#코사인 유사도 계산
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

#제목에 관한 인덱스 테이블 구축
#책 제목 입력하면 인덱스를 리턴하기 위함
indices = pd.Series(data.index, index=data['title']).drop_duplicates()


In [6]:
#입력한 책에 대해서 코사인 유사도를 이용하여 overview가 가장 유사한 10개의 영화 찾아냄
def recommendation(title, cosine_sim=cosine_sim):
    # 입력한 책의 인덱스 가져오기
    idx = indices[title]

    # 모든 영화에 대해서 해당 책과의 유사도를 구함
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 정렬
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 영화를 받아옴
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 영화의 인덱스를 받아옴
    movie_indices = [i[0] for i in sim_scores]

    #기존에 읽어들인 데이터에서 해당 인덱스의 값들을 가져온다
    #그리고 스코어 열을 추가하여 코사인 유사도도 확인할 수 있게 한다.
    result_df = data.iloc[movie_indices].copy()
    result_df['score'] = [i[1] for i in sim_scores]
    
    # 가장 유사한 10개의 영화의 제목과 스코어 리턴
    result_df = result_df[["title","score"]]
    return result_df

recommendation(book_title)

,title,score
6374,Mr. Baseball,0.244108
16285,Cooperstown,0.218530
169,Johnny Mnemonic,0.212516
19694,The Busher,0.211085
17240,Calvin Marshall,0.205888
6108,Talent for the Game,0.205088
26961,The Jackie Robinson Story,0.192555
3137,A League of Their Own,0.190057
23700,The Battered Bastards of Baseball,0.179871
20759,42,0.179753
